# Generating CSV for Tsunami.

## Method:
1. Use twitter api to get data related to a #HashTag in a time period
2. Extract relevant data(magnitude, depth, date, time) from the text retrieved
3. Clean the data
4. Convert data to CSV


## 1. Use twitter api to get data related to a HashTag in a time period

1. After creating a developer account in twitter and applying for an academic licence we get the api_key and token to access the premium services of twitter api to get all the tweets related to a #hashtag

2. We search for the #Tsunami to get all tweets related to tsunami but for our experiment we foucus on reliable tweets therefore we have only fetched tweets from the USGS twitter page by searching for @tsunamiwatch

3. We can fine-tune our search by including the date till which to search. Our period of search was 1.1.2008 to 1.1.2022

In [38]:
import tweepy
import pandas as pd
import re


# read configs
config = {}

api_key = "gU5n2ENwKk5ZitQhnFJD4LhEF"
api_key_secret = "dpqRaymjE1N3nHkQNA1k8cUo1GZtYvKoLARkBP0VjpnjymN9QW"

access_token = "941059418346602496-ppmslALuGBqeNKFEZaxpCazONh7dExI"
access_token_secret = "vX81wOWTg7hz486b5AbOftrMucN7Klx3buVjSZIqWfTTG"

# authentication
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

try:
    api = tweepy.API(auth)
    public_tweets = api.search_tweets("@tsunamiwatch", until="2022-12-15")
except:
    print("API key, secret or token is missing or incorrect") 

    


## 2. Extract relevant data from the text retrieved

The text information contains a lot of information, much of which is unnecessary. Therefore we use regular expression matching to only retieve those results that are relevant to us. 

Example:
For tsuami we look for data such as magnitude, depth, maximum water height, latitude, longitude and date-time

In [54]:
def getData(text):

    m = re.search('M[0-9][.][0-9]', text)
    d = re.search('depth [0-9][.][0-9]', text)
    lat = re.search('(\d*[.]\d)°[N|S]', text)
    long = re.search('(\d*[.]\d)°[E|W]', text)
    h = re.search('height [0-9][.][0-9]m', text)
    l = re.search('\((.*), (.*)\)', text)
    
    url = re.search('(.*)', text)
    
    magnitude=""
    depth=""
    latitude=""
    longitude=""
    location = ""
    country = ""
    maxwaterheight = ""
    
    if m:
        magnitude =  m.group(0)
        
    if lat:
        latitude = lat.group(1)
        
    if long:
        longitude = long.group(1)
        
    if d:
        depth = d.group(1)
        
    if h:
        maxwaterheight = h.group(1)
        
    if l:
        location = l.group(1)
        country = l.group(2)
        
        
    return [magnitude, depth, latitude, longitude, country, location, maxwaterheight]


## 3. Convert the data to a dataframe

After regular expression matching we have successfully extracted the relevant data.

We now combine the data we recieved from the twitter headers along with the data extracted from the tweet text and store it in a data frame.

if some information is empty then its is due to a changes in twitter's api that has truncated the text

In [55]:
# create dataframe
columns = ['Datetime','EarthquakeMagnitude','Deposits','Country','Location Name','Latitude','Longitude', 'MaximumWaterHeight']
data = []
for tweet in public_tweets:
    reldata = getData(tweet.text)
    data.append([tweet.created_at, reldata[0], 0,  reldata[4], reldata[5], reldata[2], reldata[3], reldata[6]])
    
df = pd.DataFrame(data, columns=columns)

df

,Datetime,EarthquakeMagnitude,Deposits,Country,Location Name,Latitude,Longitude,MaximumWaterHeight
0,2022-12-14 20:41:01+00:00,,0,,,,,
1,2022-12-14 19:53:07+00:00,M6.1,0,,,51.6,178.6,
2,2022-12-14 19:07:58+00:00,M6.1,0,,,51.6,178.6,
3,2022-12-14 16:49:19+00:00,M5.1,0,,,43.9,128.1,
4,2022-12-14 13:13:48+00:00,M5.1,0,,,43.9,128.1,
5,2022-12-14 13:05:21+00:00,M5.1,0,,,43.9,128.1,
6,2022-12-14 13:02:27+00:00,M5.1,0,,,43.9,128.1,


## 4. Convert data to CSV

We use one of python's built-in functions to convert a datafram to CSV file. This CSV file will be used for further processing later as mentioned in the other notebooks

In [ ]:
df.to_csv('tsunami_tweets.csv')